# Gathering valuable data from Google review using BARD api and ChatGPT api

We demonstrate the Google BARD free api only. This method is currently not usable 
Go to google bard. Then in the web page in chrome browser do the following:
- Do 'Inspect' page
- in side tab go to 'Applications'
- Go to 'cookies'
- Then go to 'https://bard.google.com' label within the 'cookies' tab
- Click on '__Secure-1PSID'

You will get your google bard API

In [1]:
import os
from bardapi import Bard

from dotenv import load_dotenv
_ = load_dotenv('bard.env') 

bard_api_key  = os.getenv('BARD_API_KEY')

In [3]:
bard = Bard(token = bard_api_key)

In [7]:
def generate_response(prompt):
    """
    Query BARD API to get response.
    """
    
    response = bard.get_answer(prompt)
                                
    
    return response

In [1]:
def generate_prompt(review_text):
    """
    Create prompt that gets sent to OpenAI API.
    """

    prompt = f'''From the following review text give me the informations 
                of overall mood of the review, good things or items and bad 
                things or items mentioned in the review. Provide the result 
                in json format with fields mood, good_items, bad_items. 
                The review text is {review_text}'''
    
    return prompt

Let us take an example from a resturant's review and check the output of the bard's answer:

In [8]:
tt = ''' 
        Been here for more than 10 times in last one year, 
        I like to visit during lunch because staff will be 
        welcoming and taste of food is really nice and night 
        times food is not great due to lot of crowd and my 
        recent experience This Saturday when I visited for dinner 
        it was over crowded as there were two birthday celebrations 
        are going on and staff was very un attentive and one of 
        the staff clearly mentioned please order only biriyani as 
        kitchen is really busy and Biriyani was not great. paneer was really nice.
        '''

In [12]:
output = generate_response(generate_prompt(tt))

In [21]:
import json

In [23]:
json.loads(output['code'])

{'mood': 'negative',
 'good_items': ['paneer'],
 'bad_items': ['service', 'crowd', 'biriyani']}

<br>
<br>

These free APIs are exhausted within a few usages. 
This is just to demonstrate that similar kind of response can be generated with Google Bard like ChatGPT


Now we move to ChatGPT again and use this tools to extarct some valuable informations about a business (A Culver's resturant at Lansing, MI).

We have 100 reviews of the resturant that are collected from Google Map Review. We will analyse which issues are affecting the bussiness in 
a bad way or good way

In [1]:
import pandas as pd

In [2]:
reviews_df = pd.read_csv('./reviews/filter650ab74699a02_csv.csv')

In [3]:
reviews_df.Review

0     This is a brand new Culver's in Lansing, Michi...
1     This is our local Culver's for South Lansing. ...
2     Extra Friendly Staff! They really went out of ...
3     Love culvers food and the flavor of the day ne...
4     This Culvers is one of the worst Culvers ive e...
                            ...                        
95    On 8/22/23 at 2:35pm I was over charged at the...
96    Just disappointed! We are traveling with trail...
97    Order was wrong and missing items.  Drive thro...
98    Absolutely love the Cod meal but the last two ...
99    Ali demonstrated amazing customer service! Foo...
Name: Review, Length: 100, dtype: object

In [4]:
import os
import openai
from dotenv import load_dotenv
_ = load_dotenv('chatgpt.env') 

openai.api_key  = os.getenv('OPENAI_API_KEY')

In [5]:
def generate_chatgpt_response(prompt):
    """
    Query OpenAI API to get response.
    """

    response = openai.ChatCompletion.create(model='gpt-3.5-turbo',
                                            temperature=0, # degree of randomness of output           
                                            messages=[{'role': 'user', 'content': prompt}]
                                           )
                                
    
    return response.choices[0].message.content


In [7]:
import time

json_response = []

for i in range(100):
    time.sleep(25)
    json_response.append(generate_chatgpt_response(generate_prompt(reviews_df.Review[i])))

In [23]:
json_response[0]

'{\n  "mood": "positive",\n  "good_items": [\n    "outstanding food",\n    "quick service",\n    "perfect temperature of food",\n    "excellent cheese curds",\n    "generous portions",\n    "polite and attentive staff",\n    "engaged and welcoming Franchisee",\n    "wonderful root beer",\n    "recommendation for dinner treat and frozen custard"\n  ],\n  "bad_items": [\n    "pricing is a little more than fast food"\n  ]\n}'

We can see that the 'good_items' or 'bad_items' are long part of sentences, which requires additional processing to find out the causes of good or bad customer experince in a quantitative way. 

In [12]:
import json

In [33]:
analysis_df = pd.DataFrame([json.loads(json_response[i]) for i in range(100)])

Moreover, the number of ways in which mood is described makes it difficult to do analysis.

In [35]:
analysis_df.mood.unique()

array(['positive', 'negative', 'neutral', 'frustrated', 'mixed',
       'Embarrassed and thankful', 'disappointed', 'disappointment',
       'Decent', 'okay', 'disappointing'], dtype=object)

### Promt Engineering: Zero shot

We can definitely see that it is difficult to do quantitative analysis with the above responses. Therefore, we would like to provide some sample format to the LLM  

In [6]:
def generate_prompt_zero(review_text):
    """
    Create prompt that gets sent to OpenAI API.
    """

    prompt = f'''From the following review text give me the informations 
                of overall mood of the review, good things or items and bad 
                things or items mentioned in the review. Mood shoud be either positive, negative or neutral.
                Good and bad things should be a single word or a list of single words.
                Provide the result in json format with fields mood, good_items, bad_items. 
                The review text is {review_text}'''
    
    return prompt

In [7]:
generate_chatgpt_response(generate_prompt_zero(reviews_df.Review[96]))

'{\n  "mood": "negative",\n  "good_items": [],\n  "bad_items": ["disappointed", "drive through only", "doors locked", "staffing"]\n}'

Still a better result ....

In [9]:
import time

json_response_zero = []

for i in range(100):
    time.sleep(25)
    json_response_zero.append(generate_chatgpt_response(generate_prompt_zero(reviews_df.Review[i])))

In [11]:
len(json_response_zero)

100

In [13]:
analysis_df0 = pd.DataFrame([json.loads(json_response_zero[i]) for i in range(100)])

Let's make a bag-of-words of the "good_items"

In [31]:
good_words = []
for i in range(100):
    splitted_words = []
    for words in analysis_df0.good_items[i] :
        splitted_words += words.split(' ')
    good_words +=splitted_words

In [33]:
from collections import Counter

In [50]:
good_dic = Counter(good_words)

In [56]:
sorted(good_dic.items(), key= lambda x:x[1])[-20:]

[('clean', 4),
 ('cooked', 4),
 ('fish', 4),
 ('the', 5),
 ('and', 6),
 ('staff', 6),
 ('burger', 6),
 ('best', 6),
 ('cheese', 7),
 ('custard', 7),
 ('great', 7),
 ('fries', 7),
 ('fresh', 7),
 ('fast', 8),
 ('with', 8),
 ('friendly', 8),
 ('hot', 9),
 ('good', 12),
 ('service', 14),
 ('food', 25)]

In [49]:
bad_words = []
for i in range(100):
    splitted_words = []
    for words in analysis_df0.bad_items[i] :
        splitted_words += words.split(' ')
    bad_words +=splitted_words

In [52]:
bad_dic = Counter(bad_words)

In [55]:
sorted(bad_dic.items(), key= lambda x:x[1])[-20:]

[('get', 6),
 ('wait', 7),
 ('waited', 7),
 ('up', 7),
 ('fries', 7),
 ('drive', 8),
 ("didn't", 8),
 ('wrong', 8),
 ('my', 8),
 ('in', 8),
 ('of', 8),
 ('service', 9),
 ('a', 9),
 ('missing', 11),
 ('no', 12),
 ('food', 13),
 ('to', 17),
 ('order', 18),
 ('the', 18),
 ('not', 22)]

## Conclussion

'Service' comes in the mist of both good and bad words.

- While 'fish', 'burger', 'cheese', 'custard' made a good impression to the customers

- 'Wait/waiting' is the most imprtant thing that made the bad reputation